In [8]:
import numpy as np

# random axis
axis_count = 20
axis_matrices = np.eye(4, dtype=np.float32)[np.newaxis,...].repeat(axis_count, axis=0)

def randomize_axis():
    #pick a position
    axis_matrices[:,3,:3] = np.random.random([axis_count,3]) * 50
    #make z aim away from center
    axis_matrices[:,2,:3] = axis_matrices[:,3,:3] / np.linalg.norm(axis_matrices[:,3,:3], axis=1).reshape([axis_count,1])
    #randomly generate x (around z)
    axis_matrices[:,0,:3] = np.random.random([axis_count,3])
    axis_matrices[:,0,:3] = axis_matrices[:,0,:3] - axis_matrices[:,2,:3] * np.einsum('ij,ij->i', axis_matrices[:,2,:3], axis_matrices[:,0,:3])[..., np.newaxis]
    axis_matrices[:,0,:3] /= np.linalg.norm(axis_matrices[:,0,:3], axis=1).reshape([axis_count,1])
    #create y
    axis_matrices[:,1,:3] = np.cross(axis_matrices[:,2,:3], axis_matrices[:,0,:3])
randomize_axis()


In [2]:
import ipywebgl

w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.enable(depth_test=True)
w.execute_commands(execute_once=True)


axis_prog = w.create_program_ext(
'''#version 300 es
    //the ViewBlock that is automatically filled by ipywebgl
    layout(std140) uniform ViewBlock
    {
        mat4 u_cameraMatrix;          //the camera matrix in world space
        mat4 u_viewMatrix;            //the inverse of the camera matrix
        mat4 u_projectionMatrix;      //the projection matrix
        mat4 u_viewProjectionMatrix;  //the projection * view matrix
    };

    uniform float u_scale;
    in vec3 in_vert;
    in vec3 in_color;
    in mat4 in_world;
    out vec3 v_color;    
    void main() {
        vec4 world =  in_world * vec4(in_vert * u_scale, 1.0);
        gl_Position = u_viewProjectionMatrix * world;
        v_color = in_color;
    }
''',
'''#version 300 es
    precision highp float;
    in vec3 v_color;
    out vec4 f_color;
    void main() {
        f_color = vec4(v_color, 1.0);
    }
''',
    
    # let's force the order of the inputs
    # in_world is a matrix so it takes 4 consecutives attributes (0,1,2,3)
    {
        'in_world' : 0,
        'in_vert' : 4,
        'in_color' : 5,
    })

axis_vbo = w.create_buffer_ext(
    src_data= np.array([
            # x, y ,z red, green, blue
            0, 0, 0, 1, 0, 0,
            1, 0, 0, 1, 0, 0,
            0, 0, 0, 0, 1, 0,
            0, 1, 0, 0, 1, 0,
            0, 0, 0, 0, 0, 1,
            0, 0, 1, 0, 0, 1,
        ], dtype=np.float32)
)


mat_vbo = w.create_buffer_ext(
    'ARRAY_BUFFER',
    axis_matrices,
    'DYNAMIC_DRAW'
)


axis_vao = w.create_vertex_array()
w.bind_vertex_array(axis_vao)
# bind the vertex buffer we want to use in this vertex array
w.bind_buffer('ARRAY_BUFFER', axis_vbo)
# enable and set the pointer to the attribute in the vertex array
w.enable_vertex_attrib_array(4)
w.vertex_attrib_pointer(4, 3, "FLOAT", False, 24, 0)
w.enable_vertex_attrib_array(5)
w.vertex_attrib_pointer(5, 3, "FLOAT", False, 24, 12)

# bind the matrix buffer
w.bind_buffer('ARRAY_BUFFER', mat_vbo)
# bind all 4 vec4 of the mat4 attribute
w.enable_vertex_attrib_array(0)
w.vertex_attrib_pointer(0, 4, "FLOAT", False, 64, 0)
w.vertex_attrib_divisor(0, 1)

w.enable_vertex_attrib_array(1)
w.vertex_attrib_pointer(1, 4, "FLOAT", False, 64, 16)
w.vertex_attrib_divisor(1, 1)

w.enable_vertex_attrib_array(2)
w.vertex_attrib_pointer(2, 4, "FLOAT", False, 64, 32)
w.vertex_attrib_divisor(2, 1)

w.enable_vertex_attrib_array(3)
w.vertex_attrib_pointer(3, 4, "FLOAT", False, 64, 48)
w.vertex_attrib_divisor(3, 1)

# execute binding commands
w.execute_commands(execute_once=True)


w.clear()
w.use_program(axis_prog)
w.uniform('u_scale', np.array([5.0], dtype=np.float32))
w.bind_vertex_array(axis_vao)
w.draw_arrays_instanced('LINES', 0, 6, axis_count)

# render in loop
w.execute_commands()
w


GLViewer(camera_pos=[0, 50, 200])

In [3]:

line_prog = w.create_program_ext(
'''#version 300 es
    //the ViewBlock that is automatically filled by ipywebgl
    layout(std140) uniform ViewBlock
    {
        mat4 u_cameraMatrix;          //the camera matrix in world space
        mat4 u_viewMatrix;            //the inverse of the camera matrix
        mat4 u_projectionMatrix;      //the projection matrix
        mat4 u_viewProjectionMatrix;  //the projection * view matrix
    };

    uniform float u_scale;
    in float in_vert;
    in mat4 in_a; 
    in mat4 in_b; 
    void main() {
        vec3 p0 = in_a[3].xyz;
        vec3 p1 = (in_a * vec4(u_scale,0,0,1)).xyz;
        vec3 p2 = (in_b * vec4(u_scale,0,0,1)).xyz;
        vec3 p3 = in_b[3].xyz;
        
        vec3 p01 =  mix(p0, p1, vec3(in_vert));
        vec3 p12 =  mix(p1, p2, vec3(in_vert));
        vec3 p23 =  mix(p2, p3, vec3(in_vert));
        
        vec3 p012 =  mix(p01, p12, vec3(in_vert));
        vec3 p123 =  mix(p12, p23, vec3(in_vert));
        
        vec3 p = mix(p012, p123, vec3(in_vert));
        gl_Position = u_viewProjectionMatrix * vec4(p, 1.0);
    }
''',
'''#version 300 es
    precision highp float;
    out vec4 f_color;
    void main() {
        f_color = vec4(.0,.0,.0, 1.0);
    }
'''
)
line_vert_count = 20
line_vbo = w.create_buffer_ext(
    src_data= np.linspace(0,1,line_vert_count, dtype=np.float32)
)

line_vao = w.create_vertex_array_ext(
    line_prog,
    [
        (line_vbo, '1f32', 'in_vert'),
        (mat_vbo, '1mat4:1 1mat4:1', 'in_a', 'in_b'),
    ]
)

w.clear()

w.use_program(axis_prog)
w.uniform('u_scale', np.array([5.0], dtype=np.float32))
w.bind_vertex_array(axis_vao)
w.draw_arrays_instanced('LINES', 0, 6, axis_count)

w.use_program(line_prog)
w.uniform('u_scale', np.array([5.0], dtype=np.float32))
w.bind_vertex_array(line_vao)
w.draw_arrays_instanced('LINE_STRIP', 0, line_vert_count, axis_count/2)

# render in loop
w.execute_commands()
w

GLViewer(camera_pos=[0, 50, 200])

In [11]:
from ipywidgets import widgets, interact

def render(p):
    
    if p==0:
        randomize_axis()
    
    center = np.sum(axis_matrices[:,3,:3], axis=0)/axis_count
    for i in range(axis_count):
        axis = axis_matrices[i,3,:3] - center
        axis /= np.linalg.norm(axis, axis=0)
        axis *= 50.0
        pos = center + axis
        axis_matrices[i,3,:3] = (pos*0.1 + axis_matrices[i,3,:3]*0.9)
    
    w.bind_buffer('ARRAY_BUFFER', mat_vbo)
    w.buffer_data('ARRAY_BUFFER', axis_matrices, 'DYNAMIC_DRAW')
    
    w.clear()

    w.use_program(axis_prog)
    w.uniform('u_scale', np.array([5.0], dtype=np.float32))
    w.bind_vertex_array(axis_vao)
    w.draw_arrays_instanced('LINES', 0, 6, axis_count)

    w.use_program(line_prog)
    w.uniform('u_scale', np.array([5.0], dtype=np.float32))
    w.bind_vertex_array(line_vao)
    w.draw_arrays_instanced('LINE_STRIP', 0, line_vert_count, axis_count/2)

    # render in loop
    w.execute_commands()

play = widgets.Play(
    value=0,
    min=0,
    max=50,
    step=1,
    interval=50,
    description="play",
    disabled=False
)

interact(render, p=play)
w

interactive(children=(Play(value=0, description='play', interval=50, max=50), Output()), _dom_classes=('widget…

GLViewer(camera_pitch=-11.799999999999999, camera_pos=[7.049012973211753, 62.72158798850099, 163.4618665704011…